## Initial

### Loading module

In [1]:
import sqlite3
import pandas as pd

#### Setting up connection to the db file

In [2]:
conn = sqlite3.connect('sample.db')
c = conn.cursor()

#### Loading the tables in pd

In [3]:
dat_ta = pd.read_sql_query('select * from transactions', conn)
dat_pd = pd.read_sql_query('select * from products', conn)
dat_sg = pd.read_sql_query('select * from segments', conn)

#### Check the data

In [4]:
dat_ta.head()

,trans_id,trans_dt,cust_id,prod_id,item_qty,item_price
0,1,2016-01-02 10:06:00,9085146,223029,1,42.99
1,2,2016-01-02 10:30:00,1215814,252270,1,103.95
2,2,2016-01-02 10:30:00,1215814,260383,1,74.99
3,4,2016-01-02 11:33:00,18511160,269119,1,51.99
4,4,2016-01-02 11:33:00,18511160,411162,1,59.99


In [5]:
dat_pd.head()

,prod_id,prod_name,brand,category
0,242151,Product 242151,Y,Make up
1,245067,Product 245067,D,Women
2,279311,Product 279311,C,Women
3,75231178,Product 75231178,C,Make up
4,218423,Product 218423,S,Women


In [6]:
dat_sg.head()

,cust_id,seg_name,update_at,active_flag
0,4402,ONE-OFFS,2014-06-01 00:00:00,N
1,4402,LAPSED,2015-12-01 00:00:00,N
2,4402,LAPSED,2015-06-01 00:00:00,N
3,4402,LAPSED,2014-01-01 00:00:00,N
4,4402,ONE-OFFS,2016-02-01 00:00:00,Y


---

## SQL questions:

### Q1:

<font color='blue'>Find the current active segment for each customer sorted by the segment update date.</font>  
<font color='blue'>The output should contain three columns:`cust_id`,`seg_name`, `updated_at`.</font>  
<font color='blue'>Here is some sample output:</font>
    
        cust_id     seg_name        updated_at
        4402        LAPSED          2014-06-01 00:00:00
        11248       ONE-OFFS        2015-10-01 00:00:00

thought: 
- WHERE: to filter active_flag, 
- Order BY: update_at

In [7]:
sql = ''' 
SELECT cust_id, seg_name, update_at 
FROM segments
WHERE active_flag = 'Y'
ORDER BY update_at
'''
data = pd.read_sql_query(sql, conn)
data.head(10)

,cust_id,seg_name,update_at
0,5764946,VIP,2014-01-01 00:00:00
1,8042053,VIP,2014-01-01 00:00:00
2,14075295,VIP,2014-01-01 00:00:00
3,15509765,VIP,2014-01-01 00:00:00
4,11259901,VIP,2014-03-01 00:00:00
5,15428328,INFREQUENT,2014-04-01 00:00:00
6,4625013,INFREQUENT,2014-11-01 00:00:00
7,7296483,VIP,2014-11-01 00:00:00
8,5744165,INFREQUENT,2014-12-01 00:00:00
9,15621121,INFREQUENT,2014-12-01 00:00:00


### Q2:
<font color='blue'>For each product purchased between Jan 2016 and May 2016 (inclusive), find
   the number of distinct transactions.</font>   
<font color='blue'>The output should contain `prod_id`,
   `prod_name` and distinct transaction columns.</font>   
<font color='blue'>Here is some sample output:</font> 

    
        prod_id     prod_name       count
        199922      Product 199922  1
        207344      Product 207344  1
        209732      Product 209732  1

thought: 
- join products and transactions table
- filter data, only including Jan 2016 - May 2016
- group by product and count the transaction time

In [8]:
sql = ''' 
SELECT p.prod_id, p.prod_name, COUNT(*) AS count
FROM products p
JOIN transactions t ON p.prod_id = t.prod_id
WHERE t.trans_dt BETWEEN '2016-01-01' AND '2016-05-31'
GROUP BY p.prod_id, p.prod_name
'''

data = pd.read_sql_query(sql, conn)
data.head(10)

,prod_id,prod_name,count
0,199922,Product 199922,1
1,207344,Product 207344,1
2,209732,Product 209732,1
3,209999,Product 209999,1
4,211200,Product 211200,1
5,211225,Product 211225,1
6,211670,Product 211670,1
7,211812,Product 211812,2
8,211857,Product 211857,1
9,212015,Product 212015,1


### Q3
<font color='blue'>Find the most recent segment of each customer as of 2016-03-01.</font>  
<font color='blue'>*Hint*: You cannot simply use `active_flag` since that is as of the current
   date *not* 2016-03-01.  </font>  
<font color='blue'>The output should contain the `cust_id`, `seg_name`
   and `update_at`  columns and should have at most one row per customer.  </font>  
<font color='blue'>Here
   is some sample output:</font>  
   
       cust_id  seg_name    update_at
       4402     ONE-OFFS    2016-02-01 00:00:00
       11248    LOYAL       2016-02-01 00:00:00
       126169   ONE-OFFS    2015-03-01 00:00:00

thought:  
- filter, keep update_at less than 2016-03-01
- group by cust_id
- select max of the update_at

In [9]:
sql = ''' 
SELECT cust_id, seg_name, MAX(update_at) 
FROM segments
WHERE update_at <= '2016-03-01'
GROUP BY cust_id
'''
data = pd.read_sql_query(sql, conn)
data.head(10)

,cust_id,seg_name,MAX(update_at)
0,4402,ONE-OFFS,2016-02-01 00:00:00
1,11248,LOYAL,2016-02-01 00:00:00
2,12064,INFREQUENT,2016-02-01 00:00:00
3,15088,ONE-OFFS,2016-02-01 00:00:00
4,66706,ONE-OFFS,2016-02-01 00:00:00
5,104497,INFREQUENT,2015-05-01 00:00:00
6,106895,LOYAL,2016-01-01 00:00:00
7,107581,ONE-OFFS,2016-02-01 00:00:00
8,116001,INFREQUENT,2015-01-01 00:00:00
9,126169,ONE-OFFS,2015-03-01 00:00:00


### Q4
<font color='blue'>Find the most popular category (by revenue) for each active segment.</font>  
<font color='blue'>*Hint*: The current (most up to date) active segment is specified by `active_flag = 'Y'` column in the segments table.</font>  
<font color='blue'>Here is the some sample output:</font>  
      	
     seg_name  category    revenue
	INFREQUENT  Women       20264

thought:
- get a sub table 
> - includes only active segment  
> - join 3 tables together  
> - compute sum of the revenue for each seg_name and category
- from this sub table, find the max revenue in each seg_name group

In [10]:
sql = ''' 

SELECT seg_name, category, MAX(revenue) AS revenue
FROM
    (SELECT s.seg_name, p.category, SUM(t.item_qty*t.item_price) AS revenue 
    FROM segments s
    JOIN transactions t ON s.cust_id=t.cust_id
    JOIN products p ON t.prod_id=p.prod_id
    WHERE s.active_flag = 'Y'
    GROUP BY s.seg_name, p.category)
GROUP BY seg_name
 
'''
data = pd.read_sql_query(sql, conn)
data

,seg_name,category,revenue
0,INFREQUENT,Women,20264.39
1,LOYAL,Women,17346.20
2,NEW,Women,3141.35
3,ONE-OFFS,Women,10690.73
4,VIP,Women,29134.07


### Q5:
<font color='blue'>Use the current sample database to find insights. </font>  
<font color='blue'>Please document your steps, include any code/documents you used (Excel, SQL, Python, etc), and have the final results in a Powerpoint format (limit 3 slides)</font>  
<font color='blue'>*Hint*: This is open ended, but you could look at sales trend, category insights, customers insights</font>  

I would like to use tableau to visualize my analysis.  
I use the code below to save the master table as customers insight in csv and import into tableau  
Please see this link  for the result  
(https://public.tableau.com/profile/chien.ming.chen#!/vizhome/Book1_15910336924430/Story1?publish=yes)

In [11]:
sql = ''' 

SELECT *, t.item_qty*t.item_price AS revenue 
FROM segments s
JOIN transactions t ON s.cust_id=t.cust_id
JOIN products p ON t.prod_id=p.prod_id
'''
data = pd.read_sql_query(sql, conn)
data.to_csv('customers insight.csv')